In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from PIL import Image




# Diretório onde as imagens estão localizadas
dataset_dir = 'dataset'

# Função para carregar e pré-processar as imagens
def load_data():
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    # Carregar os dados de treinamento e validação
    train_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='sparse',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='sparse',
        subset='validation'
    )

    return train_generator, validation_generator


# Função para construir o modelo usando ResNet50
def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(4, activation='softmax')  # 4 classes
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


# Função para treinar o modelo
def train_model(model, train_generator, validation_generator):
    history = model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator
    )
    return history


# Função para extrair características de uma imagem usando o modelo treinado
def extract_features(model, image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))  # Redimensionar para o tamanho da entrada
    img = np.expand_dims(img, axis=0) / 255.0  # Normalizar a imagem
    features = model.predict(img)
    return features.flatten()  # Retornar como vetor de características


# Função para realizar a recomendação
def recommend_image(model, image_path, feature_array, knn):
    features = extract_features(model, image_path)
    distances, indices = knn.kneighbors([features])
    return indices[0], distances[0]  # Retorna os índices e as distâncias das imagens mais próximas


# Função principal de execução
def main():
    # Carregar dados
    train_generator, validation_generator = load_data()

    # Construir o modelo
    model = build_model()

    # Treinar o modelo
    print("Treinando o modelo...")
    train_model(model, train_generator, validation_generator)

    # Extração de características de todas as imagens do dataset para recomendação
    all_image_paths = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):  # Verificando mais extensões de imagem
                all_image_paths.append(os.path.join(root, file))

    # Verifica se algum caminho de imagem foi encontrado
    if not all_image_paths:
        print(f"Aviso: Nenhum arquivo de imagem encontrado em {dataset_dir}. Por favor, verifique o caminho do diretório e as extensões dos arquivos.")
        return  # Sai do programa se nenhuma imagem for encontrada

    feature_list = []
    for img_path in all_image_paths:
        features = extract_features(model, img_path)
        feature_list.append(features)

    # Normalizar as características extraídas
    feature_array = np.array(feature_list)
    feature_array = normalize(feature_array, axis=1, norm='l2')  # Normalização L2 para evitar distâncias inconsistentes

    # Treinando o modelo K-NN para encontrar imagens semelhantes
    knn = NearestNeighbors(n_neighbors=5, metric='cosine')
    knn.fit(feature_array)

    # Testando a recomendação
    test_image = 'images1.jpg'  # Substitua pelo caminho da imagem de teste
    indices, distances = recommend_image(model, test_image, feature_array, knn)

    # Exibindo as imagens recomendadas
    print("Imagens recomendadas:")
    for i, idx in enumerate(indices):
        recommended_img_path = all_image_paths[idx]
        print(f"{i+1}. {recommended_img_path} (Distância: {distances[i]:.4f})")
        img = cv2.imread(recommended_img_path)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

if __name__ == "__main__":
    main()


Found 64 images belonging to 4 classes.
Found 13 images belonging to 4 classes.
Treinando o modelo...


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [14]:
from PIL import Image
print("Pillow version:", Image.__version__)



Pillow version: 11.1.0
